In [293]:
!pip install langchain langchain_community langchainhub openai PyPDF2 langchain_qdrant tiktoken langgraph langchain_openai

You should consider upgrading via the '/Users/felipeescobar/Documents/llm_fastapi/venv/bin/python3 -m pip install --upgrade pip' command.


In [19]:
import uuid

from langchain.chat_models.openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.retrievers.multi_vector import MultiVectorRetriever
import PyPDF2
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser
from langchain_qdrant import Qdrant
from langchain.embeddings import OpenAIEmbeddings
from langchain.storage import InMemoryByteStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import models

In [20]:
with open("protocolo-clinico-e-diretrizes-terapeuticas-do-tabagismo.pdf", "rb") as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        
        for page_num in range(len(reader.pages)):
            page = reader.pages[page_num]
            text += page.extract_text()

In [21]:
api_key = ""

In [22]:
splitter = RecursiveCharacterTextSplitter(chunk_size=4000, chunk_overlap=0)

In [23]:
docs = splitter.create_documents([text])
len(docs)


34

In [24]:
chain_summary = (
    {"doc": lambda x: x.page_content}
    | ChatPromptTemplate.from_template("Crie um resumo desse trecho:\n\n{doc}")
    | ChatOpenAI(max_retries=0, api_key =api_key)
    | StrOutputParser()
)

In [25]:
functions = [
    {
        "name": "hypothetical_questions",
        "description": "Generate hypothetical questions",
        "parameters": {
            "type": "object",
            "properties": {
                "questions": {
                    "type": "array",
                    "items": {"type": "string"},
                },
            },
            "required": ["questions"],
        },
    }
]

chain_hypothetical_query = (
    {"doc": lambda x: x.page_content}
    # Only asking for 3 hypothetical questions, but this could be adjusted
    | ChatPromptTemplate.from_template(
        "Gere uma lista de exatamente três perguntas hipotéticas que o documento abaixo poderia ser usado para responder:\n\n{doc}"
    )
    | ChatOpenAI(max_retries=0, model="gpt-4").bind(
        functions=functions, function_call={"name": "hypothetical_questions"}
    )
    | JsonKeyOutputFunctionsParser(key_name="questions")
)

In [26]:
summaries = chain_summary.batch(docs, {"max_concurrency": 5})
hypothetical_questions = chain_hypothetical_query.batch(docs, {"max_concurrency": 5})


In [27]:
qdrant_client = QdrantClient(
    api_key="woXKPNeAD8uq1fVpLLPrDMX5AM4xyjAnLDPzszQuzZKFn",
    https=False,
)

embedding_function=OpenAIEmbeddings(api_key =api_key)

doc_ids = [str(uuid.uuid4()) for _ in docs]

/Users/felipeescobar/Documents/llm_fastapi/venv/lib/python3.10/site-packages/qdrant_client/qdrant_remote.py:130: UserWarning: Api key is used with an insecure connection.
  warnings.warn("Api key is used with an insecure connection.")
/Users/felipeescobar/Documents/llm_fastapi/venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [28]:
try:
    qdrant_client.get_collection(collection_name="summaries-tgs")
except Exception as e:
    qdrant_client.create_collection(collection_name="summaries-tgs",vectors_config=models.VectorParams(
                size=1536, distance=models.Distance.COSINE
            ))
    
vectorstore = Qdrant(client=qdrant_client ,collection_name="summaries-tgs",embeddings=embedding_function)

# The storage layer for the parent documents
store = InMemoryByteStore()

id_key = "doc_id"

retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    byte_store=store,
    id_key=id_key,
)

# retriever = vectorstore.as_retriever(
#     search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.5}
# )
# retriever = vectorstore.as_retriever(
#             search_type="mmr",
#             search_kwargs={"k": 5, "lambda_mult": 0.25, "fetch_k": 50},
#         )

summary_docs = [
    Document(page_content=s, metadata={id_key: doc_ids[i]})
    for i, s in enumerate(summaries)
]

retriever.vectorstore.add_documents(summary_docs)

['a3d44e084d514aa4984876d0a52d7367',
 '9adcba62086b4d478a917ed4dd53c58c',
 '6decaa3dd2a24504900839c5e51b666d',
 '9973eb43b18c4cf6bbabdeec38d1db9d',
 '11f92dc41f974f79bb00a7941cf949c8',
 '71a7ca322db54215887ac2fb577e7368',
 'dfd66d9bbd084edd8ee1fd51ce308800',
 'd93f728c0d3949dda6a6c26c5aaf0f1b',
 '9c3453a24b9a406ba5a9919d72b59628',
 'bb1289e2101f4e5da576ef7b7cab933b',
 '09c614e4cbe4467895b31d914dc9db3f',
 '27476ad6eb7247949e6234a9857ff4e8',
 '9691d9205c434be880b5b2de45b4962b',
 '01fad766c94d474f933a9aca1b11d7cb',
 '8f89aa18d9fc42788baf5211f2d63890',
 'e03865250d9342c4bdd44fbf9ced7d26',
 '213619adbd4f4a83a41a7d7ba90de53a',
 'f407f8d5e1124841a9f4871a3966110d',
 '5e892ec775084f1783aeaa5e8723b89d',
 '71492dd4b60847bc8e1683b152ebd632',
 '389f4171dff64416aad9c37d7d7a5ce1',
 'be132b41035643d79e60dafa114e4220',
 'af3b07f182d24dbfbcf905ec2d658c05',
 '33c20e6fbdb0475c992dc76344112cab',
 'dc21a74fd0ef46bcaa77ff8e0162b12f',
 '49c1fd27e0f9478aba299a3da4ad19e5',
 'a7dc4d5e42f340e195ab70ac5b13d280',
 

In [29]:
try:
    qdrant_client.get_collection(collection_name="hypothetical-question-tgs")
except Exception as e:
    qdrant_client.create_collection(collection_name="hypothetical-question-tgs",vectors_config=models.VectorParams(
                size=1536, distance=models.Distance.COSINE
            ))
    
vectorstore = Qdrant(client=qdrant_client ,collection_name="hypothetical-question-tgs",embeddings=embedding_function)

# The storage layer for the parent documents
store = InMemoryByteStore()

id_key = "doc_id"

retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    byte_store=store,
    id_key=id_key,
)

# retriever = vectorstore.as_retriever(
#     search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.5}
# )
# retriever = vectorstore.as_retriever(
#             search_type="mmr",
#             search_kwargs={"k": 5, "lambda_mult": 0.25, "fetch_k": 50},
#         )
question_docs = []
for i, question_list in enumerate(hypothetical_questions):
    question_docs.extend(
        [Document(page_content=s, metadata={id_key: doc_ids[i]}) for s in question_list]
    )
retriever.vectorstore.add_documents(question_docs)

['0e4a64d48a7a402e9293b803f2ad4822',
 '1e02dd0d2f42420685d1b08d9649acc1',
 'c6a26c3ed5a5419ca48a01ceef90276d',
 '4f52d78429a1405bbca5a5090fd950d8',
 '08ec354cc3ea45198c9f20f1413306a5',
 '3f0816b65eec4553a55a6a4c5392dbad',
 '2ad0bc72906a438e8f605f908837ee6e',
 '3d676d02819b4a35b7b446d05ad16cd5',
 '0144e105fa7248ac87f8030bd36cb31d',
 'd6af4a45ca5c4e7993cbc77978711ae4',
 'fc8e008b5e034449a3e2e14f3dbd416a',
 '871b8c8ca819452cba891b1348bfb719',
 '00fc87fac30e418bb986fea29e81ae6f',
 'ea7f13cd3ee542fd9eb21fea1172ac1e',
 '85d285248c8c4d8fb3a78c000b15e3dd',
 '4274d8edfbc34ab8975212c09fced0b1',
 '087147dadf854a5298a5e087e5110e86',
 '600265d1463a4a26a7fcaaba9056beb0',
 '0c7828c1b00c410dadefd716a9939745',
 'bec0d5ef50a248b68b0f7ac0f8f467c0',
 '4d28f0c0e1bc4e379f2e4b5bc898ea5c',
 '106b8ec25795455ab566b17521a809c6',
 'a910bb43dc6645099f230d2b413b42a6',
 '1c535bd152e7449583ed2d5f481d5a1e',
 'd5f1644e84a842afab420cbbba0ce8a9',
 'd8a513f129024cd586d9c4d75f7277ea',
 '1e7a6b111f0e4fa1972f3249fd1db515',
 

In [30]:
retriever.docstore.mset(list(zip(doc_ids, docs)))